In [2]:
import requests 

from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from pprint import pprint
from time import sleep
import pandas as pd
import numpy as np

In [3]:
# browser = webdriver.Chrome() 
# browser.get("https://old.meneame.net/?page=129")

# # pprint(browser.page_source)
# soup = BeautifulSoup(browser.page_source, "html.parser")
# browser.close()

In [4]:
def extrae_datos_meneos (lista_meneos):
    datos = []
    continuar = True
    for meneo in lista_meneos:
        dicc_aux = {}

        try:
            dicc_aux["Comunidad"] = meneo.find("a", class_="subname").text.strip()
        except:
            dicc_aux["Comunidad"] = np.nan

        try:
            dicc_aux["Usuario"] = meneo.find("div", class_="news-submitted").find("img")["alt"]
        except:
            dicc_aux["Usuario"] = np.nan

        try:
            dicc_aux["Medio"] = meneo.find("span", class_="showmytitle").text.strip()
        except:            
            dicc_aux["Medio"] = np.nan
            
        try:
            fechas = meneo.find("div", class_="news-submitted").find_all("span", class_="ts")
            dicc_aux["Enviado"] = fechas[0].text
            dicc_aux["Publicado"] = fechas[1].text
            if len(fechas[1].text) > 5:
                fech = datetime.strptime(fechas[1].text.split(" ")[0],"%d/%m/%Y")
                if fech.year == 2022:
                    continuar = False
        except:
            dicc_aux["Enviado"] = np.nan
            dicc_aux["Publicado"] = np.nan

        try:
            dicc_aux["Titular"] = meneo.find("h2").text.strip()
        except:
            dicc_aux["Titular"] = np.nan

        try:
            dicc_aux["Entradilla"] = meneo.find("div", class_="news-content").text.strip()
        except:
            dicc_aux["Entradilla"] = np.nan
    
        try:
            dicc_aux["Votos"] = meneo.find("div", class_="votes").find("a").text
        except:
            dicc_aux["Votos"] = np.nan

        try:
            dicc_aux["Clicks"] = meneo.find("div", class_="clics").find("span").text
        except:
            dicc_aux["Clicks"] = np.nan

        try:
            dicc_aux["Positivos"] = meneo.find("span", class_="positive-vote-number").text
        except:
            dicc_aux["Positivos"] = np.nan

        try:
            dicc_aux["Anonimos"] = meneo.find("span", class_="anonymous-vote-number").text
        except:
            dicc_aux["Anonimos"] = np.nan

        try:
            dicc_aux["Negativos"] = meneo.find("span", class_="negative-vote-number").text
        except:
            dicc_aux["Negativos"] = np.nan

        try:
            dicc_aux["Comentarios"] = meneo.find("a", class_="comments")["data-comments-number"]
        except:
            dicc_aux["Comentarios"] = np.nan

        try:
            dicc_aux["Karma"] = meneo.find("span", class_="karma-number").text
        except:
            dicc_aux["Karma"] = np.nan
        
        datos.append(dicc_aux)        

    return [datos, continuar]

In [6]:
########################### BLOQUE PARA PRUEBAS ###################################
########################### Cambiar número de página y condición del contador ###################################

browser = webdriver.Chrome()
pagina = 725
lista_aux = []
contador = 0
continuar = True

while continuar == True:
    try:
        browser.get(f"https://old.meneame.net/?page={pagina}")        
        Soup_pagina_meneos = BeautifulSoup(browser.page_source, "html.parser")
        bloques_meneos = Soup_pagina_meneos.find_all("div", class_="news-summary")
        resultado = extrae_datos_meneos(bloques_meneos)    
        lista_aux.extend(resultado[0])
        continuar = resultado[1]
        pagina +=1
        contador += 1
    except:
        print(f"Error en página {pagina}")
    # sleep(random.randint(1,3))

print(f"Total paginas importadas: {contador}")

browser.close()

Total paginas importadas: 5


In [7]:
df = pd.DataFrame(lista_aux)
df

,Comunidad,Usuario,Medio,Enviado,Publicado,Titular,Entradilla,Votos,Clicks,Positivos,Anonimos,Negativos,Comentarios,Karma
0,politica,DavidEF,eldiario.es,03/01/2023 22:56,04/01/2023 02:35,"Almeida, el alcalde menguante que no para de e...","A cinco meses de las elecciones, Almeida acumu...",418,1538,146,272,22,47,449
1,actualidad,porcorosso,abc.es,03/01/2023 21:13,04/01/2023 01:55,Desalojo exprés en San Fernando de Henares: 52...,El 2023 comienza de forma amarga para los veci...,385,779,159,226,2,43,429
2,actualidad,cocolisto,lavanguardia.com,03/01/2023 19:08,04/01/2023 01:15,La variante de covid que más amenaza a Europa ...,La nueva variante XBB.1.5 del virus de la covi...,358,1527,155,203,2,47,455
3,cultura,javierchiclana,laicismo.org,03/01/2023 19:56,04/01/2023 00:45,Religiosidad declarada por edad %. Año 2022. E...,Datos CIS. Centro de investigaciones Sociológi...,314,6562,164,150,0,120,488
4,actualidad,ahotsa,elsaltodiario.com,03/01/2023 22:58,04/01/2023 00:40,Una macrocacería amenaza con diezmar la poblac...,A finales de 2021 el gobierno decía que el lob...,295,656,112,183,0,24,427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,actualidad,rojo_separatista,es.euronews.com,31/12/2022 13:37,31/12/2022 22:55,"El fantasma de la URSS persigue a los rusos, 1...","Olga Urkina, ama de casa dice: ""Yo no viví en ...",278,1750,131,147,8,490,421
121,politica,dmeijide,eldiario.es,31/12/2022 05:52,31/12/2022 21:50,"Aznar, Rajoy, Casado y Feijóo: el PP siempre b...","Con los cuatro líderes del partido al frente, ...",763,1091,262,501,11,44,404
122,cultura,falcoblau,barcelonaenhorasdeoficina.com,31/12/2022 11:57,31/12/2022 20:35,"En Barcelona hoy ha salido ""L'Home dels Nassos...","En la mitología de Catalunya, Valencia y Balea...",131,1021,69,62,17,47,436
123,actualidad,me_joneo_pensando_en_ti,motorpasion.com,31/12/2022 18:36,31/12/2022 19:45,Varias toneladas de todoterreno eléctrico y mu...,"Sin ir más lejos, Rivian Automotive tiene una ...",183,7699,108,75,8,128,485
